In [ ]:
##Vamos usar o CatBoostRegressor, então precisaremos instalar da lib catboost
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00


In [7]:
##Agora vamos importar as função train_test_split da lib scikit learn,
##A a função mean_absolute_error também do scikit, o pandas e o CatBoostRegressor(nosso modelo)
from catboost import CatBoostRegressor
import pandas as pd
from sklearn.model_selection import  train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
##Vamos carregar nosso dataset modelado:
df = pd.read_csv('/teste_indicium_precificacao.csv')

In [ ]:
##Os tipos de dados não estavam vindo como category, vou precisar alterar nesse notebook:
df[['host_id', 'nome','host_name','bairro_group','bairro', 'room_type']] = df[['host_id','nome','host_name','bairro_group','bairro', 'room_type']].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45875 entries, 0 to 45874
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   nome                           45875 non-null  category
 1   host_id                        45875 non-null  category
 2   host_name                      45875 non-null  category
 3   bairro_group                   45875 non-null  category
 4   bairro                         45875 non-null  category
 5   room_type                      45875 non-null  category
 6   price                          45875 non-null  int64   
 7   numero_de_reviews              45875 non-null  int64   
 8   calculado_host_listings_count  45875 non-null  int64   
 9   media_por_bairro               45875 non-null  float64 
 10  media_por_bairro_group         45875 non-null  float64 
 11  media_por_host_name            45875 non-null  float64 
dtypes: category(6), float64(3), int6

In [ ]:
##Vamos separar o dataset agora para começar o treinamento, removendo a coluna target('price'):
x = df.drop('price', axis = 1)
y = df['price']

In [ ]:
##Separando os dados em cojunto de treino e de teste usando o train_test_split
##Ele faz essa divisão de forma rápida e simples.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)
##Parametros passados: x, y para indicar quais dados quero dividir;
##test_size: tamanho do conjunto de teste do modelo;
##random_state: faz com que sempre que rodemos o código, a separação
##não escolha outros dados, mas sim os mesmos de antes. O valor 42 é o que normalmente todos usam como padrão;


In [ ]:
##Agora vamos criar o modelo e configurar seus parametros:
modelo = CatBoostRegressor(cat_features =['host_id','nome','host_name','bairro_group','bairro', 'room_type'], iterations= 2000, learning_rate = 0.02, depth = 10)
##cat_features: Estamos passando colunas categoricas para ele, então precisamos dizer ao modelo quais são para que ele possa fazer sua magia.
##iterations = número de vezes que o modelo pecorre os dados, melhorando sua previsão a cada iteração
##learning_rate = velocidade com que o modelo vai aprender
##depth = serve para ajudar o modelo a identificar padrões mais complexos

##Agora você pode se perguntar, por que esolhi esses valores?
##Simples, fiz vários testes até ter o melhor resultado(rsrs)!!!

In [ ]:
##Finalmente o treino:
modelo.fit(x_train, y_train)
##Passamos o conjunto de treino e demos as respectivas respostas para o modelo aprender

0:	learn: 67.6554861	total: 233ms	remaining: 7m 44s
1:	learn: 66.9082346	total: 418ms	remaining: 6m 57s
2:	learn: 66.1673724	total: 630ms	remaining: 6m 59s
3:	learn: 65.4316095	total: 852ms	remaining: 7m 4s
4:	learn: 64.7310478	total: 1.04s	remaining: 6m 55s
5:	learn: 64.0586991	total: 1.27s	remaining: 7m 2s
6:	learn: 63.3902514	total: 1.46s	remaining: 6m 55s
7:	learn: 62.7557097	total: 1.69s	remaining: 7m
8:	learn: 62.1417966	total: 1.93s	remaining: 7m 5s
9:	learn: 61.5235401	total: 2.09s	remaining: 6m 56s
10:	learn: 60.9226201	total: 2.29s	remaining: 6m 54s
11:	learn: 60.3504103	total: 2.49s	remaining: 6m 53s
12:	learn: 59.8095554	total: 2.63s	remaining: 6m 41s
13:	learn: 59.2620042	total: 2.83s	remaining: 6m 41s
14:	learn: 58.7307101	total: 2.99s	remaining: 6m 35s
15:	learn: 58.2091865	total: 3.17s	remaining: 6m 33s
16:	learn: 57.7149613	total: 3.39s	remaining: 6m 35s
17:	learn: 57.2383241	total: 3.59s	remaining: 6m 34s
18:	learn: 56.7963413	total: 3.79s	remaining: 6m 34s
19:	learn:

In [ ]:
##Vamos fazer as previsões agora usando o conjunto de teste:
##Salvaremos as previsões em uma variavel chamada y_prevs
y_prevs = modelo.predict(x_test)
##Queremos que ele faça previsôes
##Usando os dados de teste


##Agora vamos usar a métrica de avaliação mean_absolute_error para ver o desempenho:

MAE = mean_absolute_error(y_test, y_prevs) ##y test é a resposta e y_prev o que ele preveu
print(f'O mean absolute error é: {MAE:.2f}') ##Formatando para ter duas casas decimais

O mean absolute error é: 26.31


***Por que essa métrica(MAE)?***

---



Utilizai a métrica erro médio absoluto pois ela calcula a média da diferença das previsões para valores reais. Por exemplo, no nosso caso a MAE foi 26.31, o que siginifica que em média o modelo faz precificações 26.31 mais caro ou mais barato. No nosso caso é uma boa métrica pois queremos verificar o quão preciso o modelo é.

In [ ]:
##Vamos salvar como pkl
import joblib
joblib.dump(modelo, 'modelo.pkl')


['modelo.pkl']

In [8]:
####Criei uma função que modela os dados para gente, disponibilizei no meu github junto com o modelo.
def modelar_dados(dados_desejados, df_original):
  df_original = df_original[df_original['price'] <= 334]
  media_por_bairro = df_original.groupby('bairro')['price'].mean()
  media_por_bairro_group = df_original.groupby('bairro_group')['price'].mean()
  media_por_host_name = df_original.groupby('host_name')['price'].mean()

  dados_desejados['media_por_bairro'] = dados_desejados['bairro'].map(media_por_bairro)
  dados_desejados['media_por_bairro_group'] = dados_desejados['bairro_group'].map(media_por_bairro_group)
  dados_desejados['media_por_host_name'] = dados_desejados['host_name'].map(media_por_host_name)

  dados_desejados[['host_id', 'nome','host_name','bairro_group','bairro', 'room_type']] = dados_desejados[['host_id','nome','host_name','bairro_group','bairro', 'room_type']].astype('category')

  dados_desejados = dados_desejados.drop(columns = ['ultima_review', 'reviews_por_mes', 'id', 'minimo_noites', 'latitude', 'longitude', 'disponibilidade_365'], axis = 1)

  return dados_desejados

In [9]:
##Agora vamos usar o modelo no conjunto que vocês da Indicium mandaram:
##Vou fazer o upload do modelo()
import pickle
with open('/modelo.pkl', 'rb') as file:
    modelo = pickle.load(file)

##Para usar a minha função, é simples: basta carregar como dataframe em uma variavel o csv que vocês mandaram:
df = pd.read_csv('/teste_indicium_precificacao.csv')

##Agora carregar também em uma variável como dataframe os dados que querem utilizar:
dados_teste = pd.DataFrame([{'id': 2595,
 'nome': 'Skylit Midtown Castle',
 'host_id': 2845,
 'host_name': 'Jennifer',
 'bairro_group': 'Manhattan',
 'bairro': 'Midtown',
 'latitude': 40.75362,
 'longitude': -73.98377,
 'room_type': 'Entire home/apt',
 'minimo_noites': 1,
 'numero_de_reviews': 45,
 'ultima_review': '2019-05-21',
 'reviews_por_mes': 0.38,
 'calculado_host_listings_count': 2,
 'disponibilidade_365': 355}
])
##Os dados que vocês pediram que testassemos.

##Agora a função passando dois parametros: os dados que quero modelar e o df original
dados_modelados = modelar_dados(dados_teste, df)

###NÃO ESQUEÇAM DE INSTALAR OS PACOTES NECESSÁRIOS




In [10]:
##Vamos ver se funcinou
dados_modelados

,nome,host_id,host_name,bairro_group,bairro,room_type,numero_de_reviews,calculado_host_listings_count,media_por_bairro,media_por_bairro_group,media_por_host_name
0,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,Entire home/apt,45,2,181.082168,145.952835,131.534722


In [11]:
##Agora usaremos o modelo
previsao = modelo.predict(dados_modelados)
print(f'O valor previsto para esse apartamento é {previsao}')

O valor previsto para esse apartamento é [199.87239318]
